In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import copy

# === CONFIG: Optimal Hyperparameters Found by Tuning ===
BEST_PARAMS = {
    "lr": 0.005262345849736729,
    "n_layers": 3,
    "n_units_l0": 472,
    "dropout_l0": 0.4336111047085437,
    "n_units_l1": 105,
    "dropout_l1": 0.37264881799115585,
    "n_units_l2": 399,
    "dropout_l2": 0.32824162595518847,
}

EPOCHS = 200
PATIENCE = 15
BATCH_SIZE = 32
DEVICE = torch.device("cpu")

# === Load Pre-computed Features ===
print("🚀 Loading pre-computed features...")
try:
    feature_data = torch.load('extracted_features.pth')
    embeddings = feature_data['embeddings']
    labels = feature_data['labels']
    groups = feature_data['groups']
    class_names = feature_data['class_names']
    print("✅ Features loaded.")
except FileNotFoundError:
    print("❌ Error: 'extracted_features.pth' not found. Please run the Step 1 feature extraction script first.")
    exit()

# === Build the Optimal Classifier ===
layers = []
in_features = embeddings.shape[1]
for i in range(BEST_PARAMS["n_layers"]):
    out_features = BEST_PARAMS[f"n_units_l{i}"]
    layers.append(nn.Linear(in_features, out_features))
    layers.append(nn.ReLU())
    layers.append(nn.Dropout(BEST_PARAMS[f"dropout_l{i}"]))
    in_features = out_features

layers.append(nn.Linear(in_features, len(class_names)))
classifier = nn.Sequential(*layers).to(DEVICE)
print("\nOptimal classifier architecture built.")

# === K-Fold Cross-Validation ===
skf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

# Variables to store results from the best fold
best_fold_acc = 0.0
best_fold_history = None
best_fold_model_state = None
best_fold_y_true = None
best_fold_y_pred = None

for fold, (train_idx, val_idx) in enumerate(skf.split(embeddings, labels, groups)):
    print(f"\n{'='*20} FOLD {fold+1}/5 {'='*20}\n")
    
    train_dataset = TensorDataset(embeddings[train_idx], labels[train_idx])
    val_dataset = TensorDataset(embeddings[val_idx], labels[val_idx])
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    for layer in classifier.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

    optimizer = optim.Adam(classifier.parameters(), lr=BEST_PARAMS["lr"])
    criterion = nn.CrossEntropyLoss()
    
    # History for the current fold
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    best_acc_in_fold = 0.0
    epochs_no_improve = 0

    for epoch in range(EPOCHS):
        # --- Training ---
        classifier.train()
        train_loss, train_correct, train_total = 0, 0, 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            outputs = classifier(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == y_batch).sum().item()
            train_total += y_batch.size(0)
        
        history['train_loss'].append(train_loss / train_total)
        history['train_acc'].append(train_correct / train_total)

        # --- Validation ---
        classifier.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        current_preds, current_trues = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                outputs = classifier(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == y_batch).sum().item()
                val_total += y_batch.size(0)
                current_preds.extend(preds.cpu().numpy())
                current_trues.extend(y_batch.cpu().numpy())
        
        val_acc = val_correct / val_total
        history['val_loss'].append(val_loss / val_total)
        history['val_acc'].append(val_acc)
        
        # Checkpoint and Early Stopping
        if val_acc > best_acc_in_fold:
            best_acc_in_fold = val_acc
            epochs_no_improve = 0
            best_model_in_fold_state = copy.deepcopy(classifier.state_dict())
            # Store predictions from the best epoch of this fold
            best_preds_in_fold = current_preds
            best_trues_in_fold = current_trues
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"Epoch {epoch+1}: Early stopping triggered.")
                break
                
    print(f"📈 Best accuracy for fold {fold+1}: {best_acc_in_fold*100:.2f}%")
    fold_accuracies.append(best_acc_in_fold)
    
    # Check if this fold was the best one overall
    if best_acc_in_fold > best_fold_acc:
        best_fold_acc = best_acc_in_fold
        best_fold_history = history
        best_fold_y_true = best_trues_in_fold
        best_fold_y_pred = best_preds_in_fold

# === Final Results ===
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)
print(f"\n\n{'='*20} K-FOLD CROSS-VALIDATION COMPLETE {'='*20}")
print(f"Individual Fold Accuracies: {[f'{acc*100:.2f}%' for acc in fold_accuracies]}")
print(f"✅ Average Validation Accuracy: {mean_accuracy*100:.2f}% ± {std_accuracy*100:.2f}%")

# === Plotting Graphs for the Best Fold ===
print("\n📊 Generating visualizations for the best performing fold...")
plt.style.use('dark_background')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plotting Loss
ax1.plot(best_fold_history['train_loss'], label='Train Loss')
ax1.plot(best_fold_history['val_loss'], label='Validation Loss')
ax1.set_title('Loss per Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

# Plotting Accuracy
ax2.plot(best_fold_history['train_acc'], label='Train Accuracy')
ax2.plot(best_fold_history['val_acc'], label='Validation Accuracy')
ax2.set_title('Accuracy per Epoch')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

plt.tight_layout()
plt.show()

# === Plotting Confusion Matrix for the Best Fold ===
cm = confusion_matrix(best_fold_y_true, best_fold_y_pred)
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

plt.figure(figsize=(10, 8))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='viridis')
plt.title('Confusion Matrix for Best Fold')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()